# Analysis of Events per Drive and Feature Engineering in Pandas
This project aims to turn the signals gathered from the vehicle into features and prepare the data for analysis. Finally, ML models shall be build and insights generated.

## Importing Data

In [1]:
#importing necessary packages
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [122]:
#importing dataframe
df = pd.read_csv('Events_per_drive.csv')
df.head()



,VIN,Navigation active,Seat Heating on,Audio muted
0,5J8TB4H38Fl002262,1,1,0
1,5J8TB4H38Fl002262,1,0,0
2,5J8TB4H38Fl002262,1,0,1
3,5J8TB4H38Fl002262,1,0,0
4,5J8TB4H38Fl002262,0,1,1


In [22]:
df.isnull().any().any()

False

## Aggregating the data

In [117]:
#defining function to sum up values of the features in a dataframe grouped by a column and return an aggregated dataframe
def aggregate_sum(dataframe, column_name):
    df = dataframe.groupby([column_name]).sum()

    return df


#defining function to calculate ratios of values in features in a df by column -- I BUILD THIS MYSELF HAHA, TOOK 1 HOUR
def aggregate_ratio(dataframe, column_name):

    #creating placeholder list
    placeholder_lst = []

    #looping over unique values of the selected column
    for item in dataframe[column_name].unique():

        #filtering for unique value in df and storing in new df
        df = dataframe.loc[:,:][dataframe.loc[:, column_name] == item]

        placeholder_lst2 = []

        #looping over columns
        for column in df.columns:


            if column == column_name:
                placeholder_lst2.append(item)

            else:

                #calculating ratio for values that are higher than 0
                numerator = (df[column] > 0).value_counts()[1]
                denominator = df[column].count()
                ratio = round((numerator / denominator), 2)
                placeholder_lst2.append(ratio)

        placeholder_lst.append(placeholder_lst2)

    data_final = pd.DataFrame(placeholder_lst, columns = df.columns)

    return data_final





In [118]:
aggregate_ratio(df, 'VIN')

,VIN,Navigation active,Seat Heating on,Audio muted
0,5J8TB4H38Fl002262,0.6,0.53,0.6


In [28]:
for item in df['VIN'].unique():
    print(item)

5J8TB4H38Fl002262


In [18]:
df.groupby(['VIN']).mean()

,Navigation active,Seat Heating on,Audio muted
VIN,,,
5J8TB4H38Fl002262,0.6,0.533333,1.433333


In [19]:
df.groupby(['VIN']).count()

,Navigation active,Seat Heating on,Audio muted
VIN,,,
5J8TB4H38Fl002262,30,30,30


In [20]:
df.groupby(['VIN']).sum()

,Navigation active,Seat Heating on,Audio muted
VIN,,,
5J8TB4H38Fl002262,18,16,43


In [24]:
aggregate_sum(df, 'VIN')

,Navigation active,Seat Heating on,Audio muted
VIN,,,
5J8TB4H38Fl002262,18,16,43


In [58]:
list = [1,2,3]

list.append([4])

print(list)

[1, 2, 3, [4]]


In [60]:
round((2/3), 3)

0.667

In [76]:
df.columns

Index(['VIN', 'Navigation active', 'Seat Heating on', 'Audio muted'], dtype='object')

In [96]:
for columns in df.columns:
    print(columns)

VIN
Navigation active
Seat Heating on
Audio muted
